In [1]:
!pip install opencv-python vidgear tensorflow numpy Pillow


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# import required libraries
from vidgear.gears import CamGear
import cv2
import tensorflow as tf
import numpy as np
from PIL import Image as im

# Add YouTube Video URL as input source (for e.g https://youtu.be/uCy5OuSQnyA)
live_link= "https://www.youtube.com/live/gCNeDWCI0vo?si=qtOPU1vAoHqdj2kc"

MODEL_PATH = "model"
model = tf.keras.models.load_model(MODEL_PATH, compile= False)


stream = CamGear(
    source= live_link, 
    stream_mode=True,
    logging=True
).start()

# loop over
while True:

    # read frames from stream
    frame = stream.read()

    # check for frame if Nonetype
    if frame is None:
        break

    # {do something with the frame here}
    # numpy_image = frame
    # # tensorflow_image = np.reshape(numpy_image,(720, 1280, 3))
    # tensorflow_image = numpy_image
    # img = tf.convert_to_tensor(tensorflow_image, dtype=tf.float32)
    # img= cv2.imencode('.jpeg', frame)

    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), 100]  # Adjust the quality (0-100) as needed
    result, encoded_frame = cv2.imencode('.jpeg', frame, encode_param)

    # Check if the encoding was successful
    if result:
        # Convert the encoded frame to a bytes object
        jpeg_data = np.array(encoded_frame).tobytes()

    image_np = cv2.imdecode(np.frombuffer(jpeg_data, np.uint8), cv2.IMREAD_COLOR)

    # Resize the image to (384, 384) using OpenCV
    image_np = cv2.resize(image_np, (384, 384), interpolation=cv2.INTER_AREA)
    
    # Convert the NumPy array to a TensorFlow tensor and add a batch dimension
    image_tensor = tf.convert_to_tensor(image_np, dtype=tf.float32)
    image_tensor = tf.expand_dims(image_tensor, axis=0)
    # image_tensor = tf.expand_dims(image_tensor, axis=0)
    
    # img = im.fromarray(frame)
    # img= cv2.imencode('.jpeg', frame).tostring()
    # img = tf.io.decode_jpeg(image_tensor, channels = 3)
    img = tf.image.resize(image_tensor, size = (384, 384), antialias = True)
    img = img / 255.0
    
    dehazed = model(img, training = False)


    # revert
    numpy_image = dehazed.numpy()
    
    if numpy_image.max() <= 1.0:
        numpy_image *= 255.0
    
    # Step 4: Convert to OpenCV format (BGR)
    opencv_image = cv2.cvtColor(numpy_image[0], cv2.COLOR_RGB2BGR)
    opencv_image = cv2.resize(opencv_image, (960, 540))

    
    # Show output window
    cv2.imshow("Output", opencv_image)

    # check for 'q' key if pressed
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

# close output window
cv2.destroyAllWindows()

# safely close video stream
stream.stop()
